In [ ]:
using Interact

In [ ]:
using CSV, DataFrames
using Plots

In [ ]:
jhu_data = CSV.read("jhu-data.csv")
jhu_data = filter(row -> row[Symbol("Country/Region")] == "China", jhu_data)
cols = setdiff(names(jhu_data), [Symbol("Province/State"), Symbol("Country/Region"), :Lat, :Long])
jhu_data = filter(row -> row[Symbol("Province/State")] != "Hubei", jhu_data)
jhu_data = convert(Matrix, jhu_data[!, cols])'
new_cases = zeros(size(jhu_data))
K = 10
for i = 1:size(jhu_data, 1)
    new_cases[i, :] = jhu_data[i, :] .- (i > K ? jhu_data[i-K, :] : zeros(size(jhu_data, 2)))
end

In [ ]:
plot(new_cases, label="")
# conclusion: the curves look roughly normal?

In [ ]:
peak1 = 30
peak2 = 50
width1 = 20
width2 = 30
height1 = 1000
height2 = 800
days = collect(0:99)
h1 = height1 * exp.(-(days .- peak1) .^ 2 / (2 * width1 ^ 2))
h2 = height2 * exp.(-(days .- peak2) .^ 2 / (2 * width2 ^ 2))

In [ ]:
plot(days, [h1, h2])

In [ ]:
using JuMP, Gurobi

In [ ]:
function allocate_ventilators(days, h1, h2, env = Gurobi.Env(); K::Int = 1000, quad::Bool=false,
                              traveldelay::Int=1)
    model = Model(solver=GurobiSolver(env; OutputFlag=0, TimeLimit=10))
    @variable(model, x1[days] >= 0)
    @variable(model, x2[days] >= 0)
    @variable(model, travel12[days] >= 0)
    @variable(model, travel21[days] >= 0)
    @variable(model, diff1[days] >= 0)
    @variable(model, diff2[days] >= 0)
    @constraint(model, x1[days[1]] + x2[days[1]] == K)
    @constraint(model, [d = days[2:end]], x1[d] + x2[d] <= K)
    @constraint(model, travel12[days[1]] == 0)
    @constraint(model, travel21[days[1]] == 0)
    @constraint(model, [d = 2:length(days)],
                x1[days[d]] == x1[days[d-1]] - travel12[days[d]] +
                (d - traveldelay > 0 ? travel21[days[d-traveldelay]] : 0))
#     @constraint(model, [d = 2:length(days)],
#                 x1[days[d]] == x1[days[d-1]] - travel12[days[d]] + travel21[days[d-1]])
    @constraint(model, [d = 2:length(days)],
                x2[days[d]] == x2[days[d-1]] - travel21[days[d]] +
                (d - traveldelay > 0 ? travel12[days[d-traveldelay]] : 0))
    @constraint(model, [(i, d) = enumerate(days)],
                diff1[d] >= h1[i] - x1[d])
    @constraint(model, [(i, d) = enumerate(days)],
                diff2[d] >= h2[i] - x2[d])
    if quad
        @objective(model, Min, sum(diff1[d]^2 for d in days) + sum(diff2[d]^2 for d in days))
    else
        @objective(model, Min, sum(diff1[d] for d in days) + sum(diff2[d] for d in days))
    end
    solve(model)
#     @show [getvalue(travel12[d]) for d in days]
#     @show [getvalue(travel21[d]) for d in days]
    return [getvalue(x1[d]) for d in days], [getvalue(x2[d]) for d in days]
end

In [ ]:
x1, x2 = allocate_ventilators(days, h1, h2, traveldelay=2)

In [ ]:
plot(days, [min.(x1, h1) min.(x2, h2) h1 h2])

In [ ]:
x1[26:28]

In [ ]:
x2[26:28]

# Manipulating parameters

In [ ]:
env = Gurobi.Env()

In [ ]:
@manipulate for peak1 in range(20, 60, step=2), peak2 in range(20, 60, step=2),
                width1 in range(10,40,step=1), width2 in range(10,40,step=1),
                height1 in range(800,1200, step=100), height2 in range(800,1200,step=100),
                K in range(100,1000,step=100), traveldelay in 1:10, quadratic in [true, false]
    days = collect(0:99)
    h1 = height1 * exp.(-(days .- peak1) .^ 2 / (2 * width1 ^ 2))
    h2 = height2 * exp.(-(days .- peak2) .^ 2 / (2 * width2 ^ 2))
    x1, x2 = allocate_ventilators(days, h1, h2, env, K=K, quad=quadratic, traveldelay =traveldelay)
    plot(days, [min.(x1, h1) min.(x2, h2)], lw=3, ls=:dot, lc=[:blue :red], label=["Ventilators H1", "Ventilators H2"])
    plot!(days, [h1 h2], lw=3, ls=:dash, lc=[:blue :red], label=["H1 Demand" "H2 Demand"])
end